# Selecting neighborhood for a  new wine store in Toronto.

##### Miguel Tamarit, April 2020

## 1. Introduction

### 1.1 Background

##### Wine is a product often consumed at home and in restaurants.
##### Restaurants in particular have few stock space so every day they must purchase products they use. The providers of wine for restaurants must be prepared to serve their customers in short time, so they preferably must have the store near their customers.
##### On other hand, the amount of wine people buy in a store is related to population in the neighborhood, and their income amount.

### 1.2 Problem

##### Data is widely available in this times, and it is the key to find the best places for any business to locate a new shop or office 
##### This project will enable data to predict the best neighborhoods to open a new wine store.

### 1.3 Interest

##### A retail store wants to open a store in Toronto. This company sell wines to both people directly in the shop and to restaurants. 
##### They want to open their store near their potential customers. 
##### They think the best place to start business in the city will be a neighborhood with as many restaurants as possible, with more population (where presumably their products would be consumed more often), and with more average income where their more expensive (and more profitable) products will sell better. 
##### They want me to find them several neighborhoods with this criteria, but finally will be they who select the final location.

## 2. Data acquisition and cleaning

### 2.1 Data sources

##### The population and income for every Toronto neighborhood can be found in wikipedia pages https://en.wikipedia.org/wiki/Demographics_of_Toronto_neighbourhoods 
##### Some web scrapping technics will be needed to get the data from this web page. The dataset is from several years ago, but this is not a quick changing data in percentage, so it will serve for purpose.
##### On the other hand The restaurants in every neighborhood can be obtained from FourSquare using their API.

### 2.2 Data cleaning

##### The income and population  for every Toronto neighborhood are columns in the referred wikipedia web page
##### Web scrapping is the technique to get this information from the page into a usable data frame

### 2.3 Feature selection

##### Data venues for Toronto neighborhood must be filtered to get only the restaurants, and summed to get the amount of them in every neighborhood. 
##### The income and population  for every Toronto neighborhood are columns in the referred wikipedia web page. The only information to take is the next 3 columns: the name of the neighborhood, the average income, and population
##### information tables from both sources must be combined into one table.